In [ ]:
import numpy as np
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import numqi
import scipy.linalg
from utils import *

In [ ]:
H = random_hermitian(4, 0.5)
eigvals = np.linalg.eigvalsh(H)
print(H)
tr_norm = np.sum(np.abs(eigvals))
print("Trace norm of H: ", tr_norm)

In [ ]:
d = 3
theta = np.pi/2
vec_list = get_bipartite_ces(d, theta)
matrix_subspace = vec_list.reshape(-1, 2, d)
basis_orth = numqi.matrix_space.get_vector_orthogonal_basis(vec_list).reshape(-1, 2, d)
model = numqi.matrix_space.DetectCanonicalPolyadicRankModel((2,d),rank=1)
model.set_target(matrix_subspace)
kwargs = dict(theta0='uniform', num_repeat=3, tol=1e-12, print_every_round=0)
theta_optim = numqi.optimize.minimize(model,**kwargs)
print(theta_optim.fun)

In [ ]:
d = 3
theta = np.pi/4
print('Analytical GM:', analytical_GM(d, theta))
trace_norm_bound = np.sqrt(analytical_GM(d, theta))

print("Analytical trace norm bound: ", trace_norm_bound)

In [ ]:
kwargs = dict(theta0='uniform', num_repeat=3, tol=1e-12, print_every_round=0)
trace_norm_list = np.linspace(0.1, 5, 20)
N = 300

d = 3
theta0 = np.pi/2
vec_list = get_bipartite_ces(d, theta0)
min_gm_list_0 = []
for trace_norm in tqdm(trace_norm_list):
    # randomly generate hermitian matrices with chosen trace norm
    pertubation_list = [scipy.linalg.expm(-1j*random_hermitian(2*d, trace_norm)) for _ in range(N)]
    z0 = []
    for U in pertubation_list:
        pertubated_vec_list = np.array([U@vec[:,np.newaxis].reshape(2*d) for vec in vec_list])
        pertubated_matrix_subspace = pertubated_vec_list.reshape(-1, 2, d)
        model = numqi.matrix_space.DetectCanonicalPolyadicRankModel((2,d),rank=1)
        model.set_target(pertubated_matrix_subspace)
        theta_optim = numqi.optimize.minimize(model,**kwargs)
        z0.append(theta_optim.fun)
    # minimum value in z0
    min_gm_list_0.append(np.min(z0))

theta1 = np.pi/4
vec_list = get_bipartite_ces(d, theta1)
min_gm_list_1 = []
for trace_norm in tqdm(trace_norm_list):
    # randomly generate hermitian matrices with chosen trace norm
    pertubation_list = [scipy.linalg.expm(-1j*random_hermitian(2*d, trace_norm)) for _ in range(N)]
    z1 = []
    for U in pertubation_list:
        pertubated_vec_list = np.array([U@vec[:,np.newaxis].reshape(2*d) for vec in vec_list])
        pertubated_matrix_subspace = pertubated_vec_list.reshape(-1, 2, d)
        model = numqi.matrix_space.DetectCanonicalPolyadicRankModel((2,d),rank=1)
        model.set_target(pertubated_matrix_subspace)
        theta_optim = numqi.optimize.minimize(model,**kwargs)
        z1.append(theta_optim.fun)
    # minimum value in z1
    min_gm_list_1.append(np.min(z1))

theta2 = np.pi/6
vec_list = get_bipartite_ces(d, theta2)
min_gm_list_2 = []
for trace_norm in tqdm(trace_norm_list):
    # randomly generate hermitian matrices with chosen trace norm
    pertubation_list = [scipy.linalg.expm(-1j*random_hermitian(2*d, trace_norm)) for _ in range(N)]
    z2 = []
    for U in pertubation_list:
        pertubated_vec_list = np.array([U@vec[:,np.newaxis].reshape(2*d) for vec in vec_list])
        pertubated_matrix_subspace = pertubated_vec_list.reshape(-1, 2, d)
        model = numqi.matrix_space.DetectCanonicalPolyadicRankModel((2,d),rank=1)
        model.set_target(pertubated_matrix_subspace)
        theta_optim = numqi.optimize.minimize(model,**kwargs)
        z2.append(theta_optim.fun)
    # minimum value in z2
    min_gm_list_2.append(np.min(z2))





In [ ]:
# plot the minimum value of GM vs trace norm 
fig, ax = plt.subplots()
ax.plot(trace_norm_list, min_gm_list_0, 'o-', label=r'$\theta=\pi/2$')
ax.plot(trace_norm_list, min_gm_list_1, 'o-', label=r'$\theta=\pi/4$')
ax.plot(trace_norm_list, min_gm_list_2, 'o-', label=r'$\theta=\pi/6$')
#ax.set_yscale('log')
ax.set_xlabel(r'$\|H\|_{tr}$')
ax.set_ylabel(r'Minimum value of $E_2$')
ax.legend()
# save figure in pdf 
fig.savefig('robustness.pdf', bbox_inches='tight')